In [2]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

prompt_template = ChatPromptTemplate.from_template(template="你是一个数学高手，帮我解决如下的数学问题:{question}")

chain = LLMChain(llm = chat_model, prompt = prompt_template)
response = chain.invoke(input={"question":"1+2*3=?"})
print(response)

{'question': '1+2*3=?', 'text': '我们来一步一步计算这个数学表达式：\n\n表达式是：  \n$$ 1 + 2 \\times 3 $$\n\n根据**运算顺序（先乘除后加减）**，我们先计算乘法部分：\n\n$$\n2 \\times 3 = 6\n$$\n\n然后再做加法：\n\n$$\n1 + 6 = 7\n$$\n\n所以，最终结果是：\n$$\n\\boxed{7}\n$$'}


使用ChatPromptTemplate及verbose

In [4]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一个数学高手"),
    ("human","帮我解决如下的数学问题:{question}")
])

chain = LLMChain(llm = chat_model, prompt = prompt_template,verbose=True)
response = chain.invoke(input={"question":"1+2*3=?"})
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个数学高手
Human: 帮我解决如下的数学问题:1+2*3=?

> Finished chain.
{'question': '1+2*3=?', 'text': '我们按照数学中的**运算顺序（先乘除后加减）**来计算这个表达式：\n\n表达式是：  \n$$\n1 + 2 \\times 3\n$$\n\n第一步，先算乘法：  \n$$\n2 \\times 3 = 6\n$$\n\n第二步，再做加法：  \n$$\n1 + 6 = 7\n$$\n\n所以，答案是：  \n$$\n\\boxed{7}\n$$'}


SimpleSequentialChain

In [9]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains import SimpleSequentialChain

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

chain_A_template = ChatPromptTemplate.from_messages([
    ('system','你是一位精通各领域知识的知名教授'),
    ("human","请你尽可能详细的解释一下:{knowledge}")
])

chainA_chains = LLMChain(llm=chat_model,prompt=chain_A_template,verbose=True)

chainB_template = ChatPromptTemplate.from_messages([
    ("system","你非常善于提取文本中的重要信息，并做出简短的总结"),
    ("human","这是针对一个提问的完整的解释说明内容:{description}"),
    ("human","请你根据上述说明，尽可能简短的输出重要的结论，请控制在20个字以内")
])

chainB_chains = LLMChain(llm=chat_model,prompt=chainB_template,verbose=True)

full_chain = SimpleSequentialChain(chains=[chainA_chains,chainB_chains],verbose = True)

#针对于SimpleSequentialChain而言，唯一的输入的变量名是：input
response = full_chain.invoke(input={"input":"什么是langchain?"})
print(response)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你尽可能详细的解释一下:什么是langchain?

> Finished chain.
当然可以。作为一位在人工智能与自然语言处理领域深耕多年的教授，我很乐意为你全面、系统地讲解 **LangChain** 是什么，它的核心思想、架构组成、应用场景以及它为何在当前大模型（LLM）时代如此重要。

---

## 一、什么是 LangChain？

**LangChain** 是一个开源的、用于构建基于大型语言模型（Large Language Models, LLMs）的应用程序的框架。它由 Harrison Chase 在 2022 年底创建，并迅速成为最流行的 LLM 应用开发工具之一。其名称中的 “Lang” 指的是“语言”（Language），而 “Chain” 则代表“链式结构”——即通过将多个组件或步骤连接成链条来实现复杂任务。

> 📌 简单定义：
> **LangChain 是一个帮助开发者将大型语言模型与外部数据源、计算工具和业务逻辑集成起来，从而构建智能、动态、可扩展应用的开发框架。**

---

## 二、为什么需要 LangChain？——背景与动机

随着 GPT-3、GPT-4、Llama、Claude 等大模型的兴起，人们意识到这些模型具备强大的语言理解与生成能力。然而，它们也存在一些关键限制：

1. **知识截止问题（Knowledge Cut-off）**  
   大多数预训练模型的知识只更新到某个时间点（如 GPT-3.5 的知识截止于 2024 年初）。无法获取实时信息（如今天的天气、股市行情）。

2. **缺乏记忆与上下文持久化**  
   单次对话中难以维持长期记忆或用户历史行为。

3. **无法执行动作（No Action-taking Ability）**  
   模型本身不能主动调用 API、查询数据库、发送邮件等。

4. **静态输入输出模式**  
   直接调用 API 只能做简单的问答，难以支持复杂的多步推理或决策流

In [14]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains import SequentialChain

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("DASHSCOPE_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("DASHSCOPE_BASE_URL")

chat_model = ChatOpenAI(model = 'qwen-plus')

schainA_template = ChatPromptTemplate.from_messages([
    ('system','你是一位精通各领域知识的知名教授'),
    ("human","请你尽可能详细的解释一下:{knowledge}，并且{action}")
])

schainA_chains = LLMChain(llm=chat_model,prompt=schainA_template,verbose=True,output_key="schainA_chains_key")

schainB_template = ChatPromptTemplate.from_messages([
    ("system","你非常善于提取文本中的重要信息，并做出简短的总结"),
    ("human","这是针对一个提问的完整的解释说明内容:{schainA_chains_key}"),
    ("human","请你根据上述说明，尽可能简短的输出重要的结论，请控制在20个字以内")
])

schainB_chains = LLMChain(llm=chat_model,prompt=schainB_template,verbose=True,output_key='schainB_chains_key')

Seq_chain = SequentialChain(chains=[schainA_chains,schainB_chains],input_variables=["knowledge","action"],output_variables=["schainA_chains_key","schainB_chains_key"],verbose = True)

#针对于SimpleSequentialChain而言，唯一的输入的变量名是：input
response = Seq_chain.invoke({"knowledge":"中国足球为什么踢得烂","action":"举一个实际的例子"})
print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你尽可能详细的解释一下:中国足球为什么踢得烂，并且举一个实际的例子

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 你非常善于提取文本中的重要信息，并做出简短的总结
Human: 这是针对一个提问的完整的解释说明内容:作为一位长期关注体育社会学、教育体制与公共政策的教授，我将从结构性、制度性、文化性以及实践层面，对中国足球水平长期低迷的问题进行系统而深入的剖析。这个问题并非简单的“球员不行”或“教练无能”，而是根植于中国社会深层的一系列复杂因素交织而成的系统性困境。

---

### 一、结构性问题：青训体系薄弱，人才储备不足

#### 1. 青少年足球人口严重匮乏
根据国际足联（FIFA）统计数据，中国注册的青少年足球运动员约为**70万人**（2023年数据），而日本为**80万以上**，德国超过**150万**，巴西更是高达**数百万**。更关键的是，这些数字背后是参与质量的巨大差异。

在中国，大多数孩子从小被家长送往补习班而非球场。学业压力极大，学校普遍缺乏足球场地和专业教练。据教育部统计，全国中小学中拥有标准足球场的比例不足**15%**，且多数集中在城市重点学校。

#### 2. 青训体系碎片化，缺乏统一标准
中国的青训主要依赖俱乐部梯队、体校、校园足球三条路径，但三者之间缺乏有效衔接：
- 俱乐部梯队投入巨大，但只服务于少数精英；
- 体校体系陈旧，重成绩轻发展；
- 校园足球流于形式，“阳光体育”口号响亮，实际落实差。

此外，中国足协虽推出“U系列联赛”，但比赛数量少、强度低、对抗弱，难以培养真正具备职业素养的球员。

> **举例说明：武磊的成长之路**
> 武磊是中国近年来最成功的本土球员之一，曾效力西甲西班牙人队。但他成长于上海崇明岛根宝足球基地——一个由前国脚徐根宝自筹资金创办的私立青训营。这恰恰说明：**中国足球的人才不是靠国家体系培养出来的